In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.nn import Sequential, Conv2d, LeakyReLU, MaxPool2d, AdaptiveAvgPool2d, Linear, Dropout, Flatten

In [3]:
S = 7
B = 2
C = 20

In [4]:
tensor = torch.rand((S,S,B*5+C))

In [5]:
def bbox_coordinate_format(bbox):
    img_width, img_height = 448, 448

    _,x,y,w,h=bbox

    x_center = x * img_width
    y_center = y * img_height
    box_width = w * img_width
    box_height = h * img_height

    x1 = x_center - box_width / 2
    y1 = y_center - box_height / 2
    x2 = x_center + box_width / 2
    y2 = y_center + box_height / 2

    return x1.item(),y1.item(),x2.item(),y2.item()


In [6]:
for i in range(S):
    for j in range(S):
        for bbox in range(B):
            print(bbox_coordinate_format(tensor[i,j,bbox*5:(bbox+1)*5]))

(88.85006713867188, 111.79431915283203, 504.8143615722656, 164.9998321533203)
(100.5722427368164, 187.24771118164062, 215.14312744140625, 379.9302673339844)
(296.2391357421875, 238.92190551757812, 446.39215087890625, 326.5661926269531)
(-71.1300048828125, 217.91421508789062, 332.05731201171875, 530.6060791015625)
(-148.2635040283203, 74.44998168945312, 197.47996520996094, 489.3699645996094)
(-78.87001037597656, 130.9915771484375, 253.8121795654297, 329.6320495605469)
(44.41674041748047, 267.9442138671875, 140.44979858398438, 318.764892578125)
(367.408203125, 272.3364562988281, 377.4195556640625, 386.7491149902344)
(215.47935485839844, 202.93759155273438, 323.8316955566406, 628.3526611328125)
(250.73532104492188, 50.115501403808594, 301.2676696777344, 217.4642333984375)
(256.02435302734375, 51.86606979370117, 409.26751708984375, 98.83723449707031)
(117.78976440429688, 378.7978820800781, 517.010986328125, 450.2651062011719)
(332.5623779296875, 18.827857971191406, 363.22174072265625, 236.

In [7]:
def iou(bbox_predicted,bbox_actual):

    x_min_predicted,y_min_predicted,x_max_predicted,y_max_predicted=bbox_coordinate_format(bbox_predicted)
    x_min_actual,y_min_actual,x_max_actual,y_max_actual=bbox_coordinate_format(bbox_actual)

    x_intersection_right=min(x_max_actual,x_max_predicted)
    x_intersection_left=max(x_min_actual,x_min_predicted)
    y_intersection_lower=min(y_max_actual,y_max_predicted)
    y_intersection_upper=max(y_min_actual,y_min_predicted)

    w_inter=max(0,x_intersection_right-x_intersection_left)
    h_inter=max(0,y_intersection_lower-y_intersection_upper)

    w_box1=max(0,x_max_predicted-x_min_predicted)
    h_box1=max(0,y_max_predicted-y_min_predicted)

    w_box2=max(0,x_max_actual-x_min_actual)
    h_box2=max(0,y_max_actual-y_min_actual)

    area_box1=w_box1*h_box1

    area_box2=w_box2*h_box2
            
    area_inter=w_inter*h_inter

    area_union=area_box1+area_box2-area_inter

    if area_union==0:
        return 0
    else:
        iou=area_inter/area_union
        return iou

In [ ]:
def yolo_loss(pred, target, l_obj, l_nobj):
    tot_loss = 0
    for img_ind in range(len(pred)):
        for i in range(S):
            for j in range(S):
                boxes = []
                for bbox in range(B):
                    boxes.append(pred[img_ind,i,j,bbox*5:(bbox+1)*5])

                true_box = target[img_ind,i,j,0:5]
                best_box_ind = np.argmax([iou(box,true_box) for box in boxes])
                best_box = boxes[best_box_ind]

                if true_box[0]==1:
                    pred_conf = torch.sigmoid(pred[img_ind,i,j,B*5:])
                    xy_loss = torch.sum((best_box[1:3] - true_box[1:3]) ** 2)
                    wh_loss = torch.sum((torch.sqrt(torch.abs(best_box[3:5] + 1e-6)) - torch.sqrt(torch.abs(true_box[3:5] + 1e-6))) ** 2)
                    conf_loss_obj = torch.sum((pred_conf - target[img_ind,i,j,5:]) ** 2)
                    class_loss = torch.sum((best_box[0] - true_box[0]) ** 2)
                    tot_loss += l_obj * (xy_loss+wh_loss) + class_loss + conf_loss_obj
                else: 
                    class_loss = torch.sum((best_box[0] - true_box[0]) ** 2)
                    tot_loss += l_nobj * class_loss 

    return tot_loss

In [26]:
pred = torch.rand((100,S,S,B*5+C))
target = torch.rand((100,S,S,5+C))

In [27]:
loss = yolo_loss(pred,target,5,0.5)

tensor([0.4951, 0.1580, 0.8108, 0.9116, 0.6304]) [tensor([0.4951, 0.1580, 0.8108, 0.9116, 0.6304]), tensor([0.8661, 0.2273, 0.5489, 0.5058, 0.9621])]
tensor([0.4700, 0.7730, 0.5309, 0.9109, 0.8831]) [tensor([0.4700, 0.7730, 0.5309, 0.9109, 0.8831]), tensor([0.5985, 0.7326, 0.0058, 0.5904, 0.2324])]
tensor([0.0518, 0.4162, 0.5973, 0.9350, 0.2378]) [tensor([0.4292, 0.7710, 0.6404, 0.5404, 0.7936]), tensor([0.0518, 0.4162, 0.5973, 0.9350, 0.2378])]
tensor([0.4653, 0.6686, 0.9396, 0.7311, 0.0910]) [tensor([0.4653, 0.6686, 0.9396, 0.7311, 0.0910]), tensor([0.6270, 0.2865, 0.8976, 0.1943, 0.1951])]
tensor([0.0474, 0.3524, 0.0940, 0.6873, 0.8276]) [tensor([0.0474, 0.3524, 0.0940, 0.6873, 0.8276]), tensor([0.2360, 0.6091, 0.0058, 0.9063, 0.9289])]
tensor([0.5321, 0.3536, 0.6339, 0.8035, 0.6753]) [tensor([0.5321, 0.3536, 0.6339, 0.8035, 0.6753]), tensor([0.4719, 0.8281, 0.8347, 0.8290, 0.3872])]
tensor([0.6028, 0.9396, 0.7964, 0.1631, 0.0530]) [tensor([0.6028, 0.9396, 0.7964, 0.1631, 0.0530]), 

In [23]:
loss

tensor(402.7625)